In [3]:
import pandas as pd
from sqlalchemy import create_engine, text
import cred_pg as c
import psycopg2

In [4]:
engine = create_engine(
    f'postgresql://{c.pg_userid}:{c.pg_password}@{c.pg_host}/{c.pg_db}', 
    connect_args = {
        'options': '-c search_path=${user},ugeobln,ugm,uinsta,umisc,umobility,usozmed,public', 
        'keepalives_idle': 120
    },
    pool_size=1, 
    max_overflow=0,
    execution_options={ 'isolation_level': 'AUTOCOMMIT' }
)

In [1]:
def bsp_er_diagram():
    return """
       

        @startuml
        ' hide the spot
        hide circle

        ' avoid problems with angled crows feet
        skinparam linetype ortho
    entity "linie" as linie {
  *lid : integer not null
  *bez : character varying not null
}
entity "abschnitt" as abschnitt {
  *ulid : integer not null
  *nr : integer not null
  *hid_a : integer not null
  *hid_b : integer not null
  *haelt : character not null
}
entity "haltestelle" as haltestelle {
  *hid : integer not null
  *bez : character varying not null
  *lat : double precision not null
  *lng : double precision not null
  *pos : USER-DEFINED 
  *posp : USER-DEFINED 
}
entity "fahrt" as fahrt {
  *fid : bigint not null
  *zpid : integer not null
  *ulid : integer not null
}
entity "segment" as segment {
  *hid_a : integer not null
  *hid_b : integer not null
  *laenge_in_meter : integer 
}
entity "unterlinie" as unterlinie {
  *ulid : integer not null
  *lid : integer not null
}
entity "halt" as halt {
  *haid : bigint not null
  *fid : bigint not null
  *hid : integer not null
  *nr : integer not null
  *zeit_ankunft : time without time zone 
  *zeit_abfahrt : time without time zone 
}
entity "geo" as geo {
  *osm_id : character varying not null
  *code : integer not null
  *fclass : character varying not null
  *name : character varying 
  *wkt : text 
  *shape : USER-DEFINED 
  *shapep : USER-DEFINED 
}
entity "zeitplan" as zeitplan {
  *zpid : integer not null
  *datum_beginn : date not null
  *datum_ende : date not null
  *mo : integer not null
  *di : integer not null
  *mi : integer not null
  *do : integer not null
  *fr : integer not null
  *sa : integer not null
  *so : integer not null
}

     
zeitplan ||--o{ fahrt  : ZeitPlan | *ZPID
segment }|..|| haltestelle : Segment | *HID A, *HID B
halt ||--o{ fahrt : Halt | *FID
segment }|..|| haltestelle : Halt | *HID
abschnitt }|..|| unterlinie : Abschnitt | *ULID
unterlinie }|..|| linie : Unterlinie | *LID
abschnitt ||--o{  haltestelle :Abschnitt | HID A  HID B
unterlinie }o..o| fahrt : Unterlinie | *FID
abschnitt ||--o{  haltestelle :Abschnitt | HID A  HID B


        @enduml
    


    """

print(bsp_er_diagram())


       

        @startuml
        ' hide the spot
        hide circle

        ' avoid problems with angled crows feet
        skinparam linetype ortho
    entity "linie" as linie {
  *lid : integer not null
  *bez : character varying not null
}
entity "abschnitt" as abschnitt {
  *ulid : integer not null
  *nr : integer not null
  *hid_a : integer not null
  *hid_b : integer not null
  *haelt : character not null
}
entity "haltestelle" as haltestelle {
  *hid : integer not null
  *bez : character varying not null
  *lat : double precision not null
  *lng : double precision not null
  *pos : USER-DEFINED 
  *posp : USER-DEFINED 
}
entity "fahrt" as fahrt {
  *fid : bigint not null
  *zpid : integer not null
  *ulid : integer not null
}
entity "segment" as segment {
  *hid_a : integer not null
  *hid_b : integer not null
  *laenge_in_meter : integer 
}
entity "unterlinie" as unterlinie {
  *ulid : integer not null
  *lid : integer not null
}
entity "halt" as halt {
  *haid : bigint no

In [30]:
def emit_start():
    return """
        @startuml
        ' hide the spot
        hide circle

        ' avoid problems with angled crows feet
        skinparam linetype ortho
    """

def emit_end():
    return """
        @enduml
    """

def emit_entity(table_name, columns):
    entity = f'entity "{table_name}" as {table_name} {{\n'
    for column in columns:
        data_type = column["data_type"]
        not_null = "not null" if column["is_nullable"] == "NO" else ""
        entity += f'  *{column["column_name"]} : {data_type} {not_null}\n'
    entity += '}\n'
    return entity

def emit_entities(schema):
    with engine.connect() as con:
        sql = f"""
        SELECT table_name
        FROM information_schema.tables
        WHERE table_schema = '{schema}' AND table_type = 'BASE TABLE'
        """
        df_tables = pd.read_sql_query(text(sql), con)

        entities = ""
        for table in df_tables['table_name']:
            sql = f"""
            SELECT column_name, data_type, is_nullable
            FROM information_schema.columns
            WHERE table_name = '{table}' AND table_schema = '{schema}'
            """
            df_columns = pd.read_sql_query(text(sql), con)
            entities += emit_entity(table, df_columns.to_dict('records'))

        return entities
def emit_relationships(schema):
  with engine.connect() as con:
    sql = f"""

        SELECT
            conname AS constraint_name,
            conrelid::regclass AS table_name,
            a.attname AS column_name,
            confrelid::regclass AS primary_table,
            af.attname AS primary_column
        FROM pg_constraint AS c
        JOIN pg_namespace AS n ON n.oid = c.connamespace
        JOIN pg_class AS cl ON cl.oid = c.conrelid
        JOIN pg_class AS clt ON clt.oid = c.confrelid
        JOIN pg_attribute AS a ON a.attnum = ANY(c.conkey) AND a.attrelid = c.conrelid
        JOIN pg_attribute AS af ON af.attnum = ANY(c.confkey) AND af.attrelid = c.confrelid
        WHERE n.nspname = '{schema}' AND c.contype = 'f';
        """
    
    df_relationships = pd.read_sql_query(text(sql), con)

    relationships = ""
  for _, row in df_relationships.iterrows():
            
            relationship_symbol = "-->"
            
           # relationships += f'{row["table_name"]} "1" --o "{row["primary_table"]}" "n" : {row["foreign_column"]} -- {row["primary_column"]}\n'
            relationships += f'{row["table_name"]} {relationship_symbol} {row["primary_table"]} : {row["column_name"]}\n'

  return relationships

  
     
#      return """
#      beziehung  umkher 
# zeitplan ||--o{ fahrt  : ZeitPlan | *ZPID
# segment }|..|| haltestelle : Segment | *HID A, *HID B
# halt ||--o{ fahrt : Halt | *FID
# segment }|..|| haltestelle : Halt | *HID
# abschnitt }|..|| unterlinie : Abschnitt | *ULID
# unterlinie }|..|| linie : Unterlinie | *LID
# abschnitt ||--o{  haltestelle :Abschnitt | HID A  HID B
# unterlinie }o..o| fahrt : Unterlinie | *FID
# abschnitt ||--o{  haltestelle :Abschnitt | HID A  HID B

# """

def er_diagram(schema):
    start = emit_start()
    entities = emit_entities(schema)
    relationships = emit_relationships(schema)
    end = emit_end()
    return f"{start}{entities}{relationships}{end}"

print(er_diagram("umobility"))



        @startuml
        ' hide the spot
        hide circle

        ' avoid problems with angled crows feet
        skinparam linetype ortho
    entity "linie" as linie {
  *lid : integer not null
  *bez : character varying not null
}
entity "abschnitt" as abschnitt {
  *ulid : integer not null
  *nr : integer not null
  *hid_a : integer not null
  *hid_b : integer not null
  *haelt : character not null
}
entity "haltestelle" as haltestelle {
  *hid : integer not null
  *bez : character varying not null
  *lat : double precision not null
  *lng : double precision not null
  *pos : USER-DEFINED 
  *posp : USER-DEFINED 
}
entity "fahrt" as fahrt {
  *fid : bigint not null
  *zpid : integer not null
  *ulid : integer not null
}
entity "segment" as segment {
  *hid_a : integer not null
  *hid_b : integer not null
  *laenge_in_meter : integer 
}
entity "unterlinie" as unterlinie {
  *ulid : integer not null
  *lid : integer not null
}
entity "halt" as halt {
  *haid : bigint not null
  